In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dsn-multilingual-data/test.xlsx
/kaggle/input/dsn-multilingual-data/train.xlsx


In [3]:
!pip install -q -U transformers==4.55.2 trl==0.21.0 datasets==4.0.0 huggingface_hub==0.34.4 accelerate==1.10.0 peft==0.17.0 bitsandbytes==0.47.0 wandb==0.21.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 100.3 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 90.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:010

In [4]:
import os
import torch
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, setup_chat_format
from transformers import BitsAndBytesConfig

2025-10-23 05:43:17.539345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761198197.927256      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761198198.036516      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

hf_token = UserSecretsClient().get_secret("HF_TOKEN")

login(token=hf_token)

In [6]:
# MODEL_NAME = "CohereLabs/aya-23-8B"
MODEL_NAME = "facebook/nllb-200-distilled-600M"
# NEW_MODEL_REPO = "Yanmife/aya-wiki-data"
NEW_MODEL_REPO = "Yanmife/nllb-nigeria"

WANDB_PROJECT = "Fine-tuning-nllb-nigeria"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [8]:
def get_tokenizer_for_lang(lang_code):
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        src_lang=lang_code,
        tgt_lang="eng_Latn",
        trust_remote_code=True
    )
    return tokenizer

yoruba_tokenizer = get_tokenizer_for_lang("yor_Latn")
igbo_tokenizer = get_tokenizer_for_lang("ibo_Latn")
hausa_tokenizer = get_tokenizer_for_lang("hau_Latn")

In [9]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=False,
    low_cpu_mem_usage=True,
)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [10]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,            # rank
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

In [11]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 618,612,736 || trainable%: 0.5721


In [12]:
train_df = pd.read_excel("/kaggle/input/dsn-multilingual-data/train.xlsx")
test_df = pd.read_excel("/kaggle/input/dsn-multilingual-data/test.xlsx")

In [13]:
test_df.head(10)

,Competition_ID,Input Text,Language
0,IGB001,Onye ọ bụla tukotara ego iji fu na emeziri ihe...,Igbo
1,IGB002,Anyị bughariri ọrụ ụgwọ metara nile iji debe i...,Igbo
2,IGB003,Emeputara obere akwụkwọ ndekọ ka anyị wee nwee...,Igbo
3,IGB004,Anyị kwekọrịtara ka onye ọ bụla kwụọ ụgwọ ọnụ ...,Igbo
4,IGB005,Echetaram ha na-ntunye ụtụ imezi ihe nke ọma n...,Igbo
5,IGB006,Anyị kwekọrịtara ka- ekewa uloahia mkpuruosisi...,Igbo
6,IGB007,A kwadorom okputoro mkpu ntọala maka ịtụ ego m...,Igbo
7,IGB008,Anyị na- agbanwe ikwu ụgwọ maka mficha n' ọnwa...,Igbo
8,IGB009,Onye ọ bụla n'ime ndị bi n' otu ụlọ na- atuta ...,Igbo
9,IGB010,A gwara m ha ka anyị chigha azụ n' otu esi eke...,Igbo


In [14]:
train_df.head()

,Output,input,Language
0,"So, I find myself, over and over again, thinki...",оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
1,Especially in things where the connection to G...,"Karịsịa na ihe ebe na njikọ aka Chineke otuto,...",Igbo
2,"12 , 13 . ( a ) What is hyperbole ?\n","12 , 13 . ( a ) Kí ni àbùmọ́ ?\n",Yoruba
3,You and your story have helped me.\n,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
4,CAUSE ALL PEOPLE TO BE TREATED EQUALLY,NaN,Igbo


In [15]:
print("Train size per language:")
print(train_df['Language'].value_counts())

print("\nTest size per language:")
print(test_df['Language'].value_counts())

Train size per language:
Language
Yoruba    45055
Igbo      45001
Hausa     44944
Name: count, dtype: int64

Test size per language:
Language
Hausa     229
Yoruba    200
Igbo      168
Name: count, dtype: int64


In [16]:
train_df[train_df["Language"] == "Hausa"].head(20)

,Output,input,Language
0,"So, I find myself, over and over again, thinki...",оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
3,You and your story have helped me.\n,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
5,"‘Assuredly, I say to you, I do not know you.’\n",Kuna tsammani na haƙa ne kuma ban san abin da ...,Hausa
6,"""Public and community radio stations are frequ...",оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
23,Being an American in Japan brings a lot of ben...,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
28,INTR2016 US Foreign and Security Policy in Asia\n,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
32,(1) Christians in America and in the West simp...,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
33,Q: Tell us about your future activities in Mal...,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
38,Removing Military Weapons From Our Communities\n,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa
39,He has been very good to me and his family is ...,оооооооооооооооооооооооооооооооооооооооооооооо...,Hausa


In [17]:
import re
import pandas as pd

def is_corrupted(text):
    if not isinstance(text, str):
        return True
    
    # Detect non-Latin characters (Cyrillic, etc.)
    non_latin = bool(re.search(r'[^\u0000-\u024F\s]', text))
    # Detect long repeated sequences like "oooooo..." (Cyrillic or Latin)
    repeated_o = bool(re.search(r'(о|o|O|О){10,}', text))
    # Detect text that is mostly made of a single character
    too_repetitive = len(set(text.strip())) < 3

    return non_latin or repeated_o or too_repetitive


def corruption_summary(df):
    summary = []
    for lang, group in df.groupby("Language"):
        corrupted_count = group["input"].apply(is_corrupted).sum()
        total = len(group)
        summary.append({
            "Language": lang,
            "corrupted": corrupted_count,
            "total": total,
            "%_corrupted": round((corrupted_count / total) * 100, 2)
        })
    return pd.DataFrame(summary).set_index("Language")


# Example usage
summary_df = corruption_summary(train_df)
print("=== Corruption Summary by Language ===")
print(summary_df)


=== Corruption Summary by Language ===
          corrupted  total  %_corrupted
Language                               
Hausa         40324  44944        89.72
Igbo          42915  45001        95.36
Yoruba        42327  45055        93.95


In [18]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": test_dataset  # use test as validation if no val split
})
dataset

DatasetDict({
    train: Dataset({
        features: ['Output', 'input', 'Language'],
        num_rows: 135000
    })
    validation: Dataset({
        features: ['Competition_ID', 'Input Text', 'Language'],
        num_rows: 597
    })
})

In [19]:
dataset["train"][1]

{'Output': "Especially in things where the connection to God's glory isn't as clear.",
 'input': 'Karịsịa na ihe ebe na njikọ aka Chineke otuto, ọ dịghị ka o doo anya.',
 'Language': 'Igbo'}

In [20]:
import re
import pandas as pd

def is_corrupted(text):
    """Detects corrupted or invalid text samples."""
    if not isinstance(text, str):
        return True

    text = text.strip()
    if len(text) < 3:  # too short
        return True

    # Non-Latin + non-Yoruba character sequences (detect Cyrillic, Chinese, etc.)
    if re.search(r'[^\u0000-\u024F\u1E00-\u1EFF\s\'’.,!?-]', text):
        return True

    # Excessive repetition of same character (e.g. oooooooo)
    if re.search(r'(.)\1{9,}', text):
        return True

    # Mostly one repeated character or empty after stripping punctuation
    stripped = re.sub(r'[\W_]+', '', text)
    if len(set(stripped)) < 3:
        return True

    # Too many digits or symbols compared to letters
    letters = len(re.findall(r'[A-Za-z]', text))
    non_letters = len(re.findall(r'[^A-Za-z\s]', text))
    if non_letters > letters:
        return True

    # Detect mix of scripts (e.g. Cyrillic + Latin)
    if re.search(r'[А-Яа-я]', text) and re.search(r'[A-Za-z]', text):
        return True

    return False


def corruption_summary(df, text_col="input"):
    """Summarizes corruption rate per language."""
    summary = []
    for lang, group in df.groupby("Language"):
        corrupted_mask = group[text_col].apply(is_corrupted)
        corrupted_count = corrupted_mask.sum()
        total = len(group)
        summary.append({
            "Language": lang,
            "corrupted": corrupted_count,
            "total": total,
            "%_corrupted": round((corrupted_count / total) * 100, 2)
        })
    return pd.DataFrame(summary).set_index("Language")


def clean_dataset(df, text_col="input"):
    """Removes corrupted rows and returns clean DataFrame."""
    corrupted_mask = df[text_col].apply(is_corrupted)
    clean_df = df[~corrupted_mask].reset_index(drop=True)
    dropped = corrupted_mask.sum()
    print(f"✅ Dropped {dropped} corrupted rows ({(dropped / len(df)) * 100:.2f}%).")
    return clean_df

summary_before = corruption_summary(train_df)
print("=== Corruption Summary (Before Cleaning) ===")
print(summary_before)

# clean
train_df_clean = clean_dataset(train_df)

# summary after cleaning
summary_after = corruption_summary(train_df_clean)
print("\n=== Corruption Summary (After Cleaning) ===")
print(summary_after)

=== Corruption Summary (Before Cleaning) ===
          corrupted  total  %_corrupted
Language                               
Hausa         40135  44944        89.30
Igbo          16350  45001        36.33
Yoruba        42466  45055        94.25
✅ Dropped 98951 corrupted rows (73.30%).

=== Corruption Summary (After Cleaning) ===
          corrupted  total  %_corrupted
Language                               
Hausa             0   4809          0.0
Igbo              0  28651          0.0
Yoruba            0   2589          0.0


In [21]:
train_df_clean

,Output,input,Language
0,Especially in things where the connection to G...,"Karịsịa na ihe ebe na njikọ aka Chineke otuto,...",Igbo
1,Be loyal .,Kwụsie ike n’akụkụ enyi gị .,Igbo
2,A lack of concentration is a common complaint ...,A enweghi ntinye bụ na-ekwukarị ụbọchị ndị a.,Igbo
3,* That’s how we arrive at 1914 as the year tha...,* Otú a ka anyị si mata na oge asaa ahụ gwụrụ ...,Igbo
4,My parents and the house I shared with them an...,Papa m na mama m na ụlọ mụ na ha na ụmụnne m a...,Igbo
...,...,...,...
36044,They forfeited their good standing with Jehova...,"Ha tụfuru ezi mmekọrịta ha na Jehova nwere , t...",Igbo
36045,Why did Jesus give those two parables ?\n,Kí nìdí tí Jésù fi lo àwọn àpèjúwe méjèèjì yẹn...,Yoruba
36046,"Then, utter fear (and gravity) takes over.\n",A taƙaice taƙaitacce.\n,Hausa
36047,For Allah is well-acquainted with all that ye ...,"Kuma abin da kuka aikata daga alhħri, Allah Ya...",Hausa


In [22]:
train_df_clean["Language"].value_counts()

Language
Igbo      28651
Hausa      4809
Yoruba     2589
Name: count, dtype: int64

In [23]:
train_dataset = Dataset.from_pandas(train_df_clean)
train_dataset

Dataset({
    features: ['Output', 'input', 'Language'],
    num_rows: 36049
})

In [24]:
train_dataset[1]

{'Output': 'Be loyal .',
 'input': 'Kwụsie ike n’akụkụ enyi gị .',
 'Language': 'Igbo'}

In [25]:
opus_ha = load_dataset("opus100", "en-ha")
opus_ha

README.md: 0.00B [00:00, ?B/s]

en-ha/test-00000-of-00001.parquet:   0%|          | 0.00/244k [00:00<?, ?B/s]

en-ha/train-00000-of-00001.parquet:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

en-ha/validation-00000-of-00001.parquet:   0%|          | 0.00/246k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/97983 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 97983
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [26]:
opus_ha["train"][0]

{'translation': {'en': '(O Prophet), tell those of My servants who believe that they should establish Prayer and spend out of what We have provided them with, both secretly and openly, before there arrives the Day when there will be no bargaining, nor any mutual befriending.',
  'ha': 'Ka ce wa bãyĩNa waɗanda suka yi ĩmãni su tsayar da salla kuma su ciyar daga abin da Muka azurtã su, a asirce da bayyane, daga gabãnin wani wuni ya zo, bãbu ciniki a ciki, kuma bãbu abõtaka.'}}

In [27]:
opus_ig = load_dataset("opus100", "en-ig")
opus_ig

en-ig/test-00000-of-00001.parquet:   0%|          | 0.00/44.4k [00:00<?, ?B/s]

en-ig/train-00000-of-00001.parquet:   0%|          | 0.00/770k [00:00<?, ?B/s]

en-ig/validation-00000-of-00001.parquet:   0%|          | 0.00/45.3k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1843 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18415 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1843 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 1843
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 18415
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1843
    })
})

In [28]:
ig_mt_data = concatenate_datasets([
    opus_ig["train"],
    opus_ig["test"],
    opus_ig["validation"]
])

print(ig_mt_data)
print(f"Total samples: {len(ig_mt_data)}")

# Optional: peek at a few examples
print(ig_mt_data[0])

Dataset({
    features: ['translation'],
    num_rows: 22101
})
Total samples: 22101
{'translation': {'en': '25%', 'ig': '25%'}}


In [29]:
ig_mt_data[10]

{'translation': {'en': 'Default zoom level used by the icon view.',
  'ig': 'Difọọltụ ógó ndapụ ọsịṣọ nke ngosi aịkọn jiri rụọ ọrụ.'}}

In [30]:
!git clone https://github.com/uds-lsv/menyo-20k_MT.git

Cloning into 'menyo-20k_MT'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 151 (delta 59), reused 71 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (151/151), 4.43 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [31]:
data_path = "menyo-20k_MT/data"

# Load train/dev/test
menyo_ds = load_dataset(
    "csv",
    data_files={
        "train": os.path.join(data_path, "train.tsv"),
        "dev": os.path.join(data_path, "dev.tsv"),
        "test": os.path.join(data_path, "test.tsv"),
    },
    delimiter="\t",  # important: TSV files
)

print(menyo_ds)

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['English', 'Yoruba'],
        num_rows: 10070
    })
    dev: Dataset({
        features: ['English', 'Yoruba'],
        num_rows: 3397
    })
    test: Dataset({
        features: ['English', 'Yoruba'],
        num_rows: 6633
    })
})


In [32]:
yor_mt_data = concatenate_datasets([menyo_ds["train"], menyo_ds["test"], menyo_ds["dev"]])
yor_mt_data

Dataset({
    features: ['English', 'Yoruba'],
    num_rows: 20100
})

In [33]:
yor_mt_data[1]

{'English': 'This work is licensed under a Creative Commons Attribution 4.0 International License.',
 'Yoruba': 'Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons Attribution 4.0 International License.'}

In [37]:
import json
import pandas as pd
import random

def combine_datasets(yoruba_ds, hausa_ds, hackathon_ds, seed=42):
    random.seed(seed)

    def clean_text(text):
        if not isinstance(text, str):
            return ""
        text = text.strip()
        text = " ".join(text.split())
        return text

    def safe_load(item):
        """Safely load JSON strings or pass through dicts."""
        if isinstance(item, str):
            try:
                return json.loads(item)
            except json.JSONDecodeError:
                return {}
        return item if isinstance(item, dict) else {}

    combined = []

    # === Yoruba (Menyo20k) ===
    for sample in yoruba_ds:
        sample = safe_load(sample)
        src = clean_text(sample.get("Yoruba", ""))
        tgt = clean_text(sample.get("English", ""))
        if src and tgt:
            combined.append({
                "src_text": f"<yor> {src}",
                "tgt_text": tgt,
                "language_tag": "yor"
            })

    hausa_samples = list(hausa_ds)
    random.shuffle(hausa_samples)
    hausa_samples = hausa_samples[:20000]  
    
    for sample in hausa_samples:
        sample = safe_load(sample)
        tr = safe_load(sample.get("translation", {}))
        src = clean_text(tr.get("ha", ""))
        tgt = clean_text(tr.get("en", ""))
        if src and tgt:
            combined.append({
                "src_text": f"<hau> {src}",
                "tgt_text": tgt,
                "language_tag": "hau"
            })

    for sample in hackathon_ds:
        sample = safe_load(sample)
        lang = clean_text(sample.get("Language", "")).lower()
        src = clean_text(sample.get("input", ""))
        tgt = clean_text(sample.get("Output", ""))
        lang_tag = None

        if lang in ["igbo", "ibo"]:
            lang_tag = "ibo"
        elif lang == "yoruba":
            lang_tag = "yor"
        elif lang == "hausa":
            lang_tag = "hau"

        if src and tgt and lang_tag:
            combined.append({
                "src_text": f"<{lang_tag}> {src}",
                "tgt_text": tgt,
                "language_tag": lang_tag
            })

    df = pd.DataFrame(combined).drop_duplicates(subset=["src_text", "tgt_text"]).reset_index(drop=True)
    return df


In [38]:
concatenated_dataset = combine_datasets(yor_mt_data, opus_ha["train"], train_dataset)
concatenated_dataset.head()

,src_text,tgt_text,language_tag
0,<yor> ﻿Ìdá 1: Kín ni Creative Commons?,Unit 1: What is Creative Commons?,yor
1,<yor> Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons A...,This work is licensed under a Creative Commons...,yor
2,<yor> Creative Commons jẹ́ àwọn ọ̀kan-ò-jọ̀kan...,"Creative Commons is a set of legal tools, a no...",yor
3,<yor> Creative Commons bẹ̀rẹ̀ láti wá wọ̀rọ̀kọ...,Creative Commons began in response to an outda...,yor
4,<yor> Àwọn àṣẹ CC jẹ mọ́ àṣẹ ẹni tí ó ní iṣẹ́-...,CC licenses are built on copyright and are des...,yor


In [39]:
print(f"✅ Total combined rows: {len(concatenated_dataset)}")
print("Language distribution:\n", concatenated_dataset["language_tag"].value_counts())

✅ Total combined rows: 73060
Language distribution:
 language_tag
ibo    26712
hau    24159
yor    22189
Name: count, dtype: int64


In [42]:
full_dataset = Dataset.from_pandas(concatenated_dataset)
full_dataset.shuffle(seed=42)


Dataset({
    features: ['src_text', 'tgt_text', 'language_tag'],
    num_rows: 73060
})

In [43]:
full_dataset[1]

{'src_text': '<yor> Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons Attribution 4.0 International License.',
 'tgt_text': 'This work is licensed under a Creative Commons Attribution 4.0 International License.',
 'language_tag': 'yor'}

In [44]:
from datasets import Dataset

LANG_CODE_MAP = {
    "Yoruba": "yor_Latn",
    "Igbo": "ibo_Latn",
    "Hausa": "hau_Latn"
}

def add_lang_codes_hf(dataset, input_col="input", lang_col="Language"):

    def prepend_code(example):
        lang_name = example.get(lang_col, "")
        code = LANG_CODE_MAP.get(lang_name, "")
        text = example.get(input_col, "")
        if not isinstance(text, str):
            text = ""
        example[input_col] = f"{code} {text}".strip()
        return example

    return dataset.map(prepend_code)

In [45]:
full_datasets = add_lang_codes_hf(full_dataset)
full_datasets

Map:   0%|          | 0/73060 [00:00<?, ? examples/s]

Dataset({
    features: ['src_text', 'tgt_text', 'language_tag', 'input'],
    num_rows: 73060
})

In [46]:
full_datasets[1]

{'src_text': '<yor> Iṣẹ́ yìí wà lábẹ́ àṣẹ Creative Commons Attribution 4.0 International License.',
 'tgt_text': 'This work is licensed under a Creative Commons Attribution 4.0 International License.',
 'language_tag': 'yor',
 'input': ''}

In [47]:
split_ds = full_datasets.train_test_split(test_size=0.05, seed=42)
split_ds

DatasetDict({
    train: Dataset({
        features: ['src_text', 'tgt_text', 'language_tag', 'input'],
        num_rows: 69407
    })
    test: Dataset({
        features: ['src_text', 'tgt_text', 'language_tag', 'input'],
        num_rows: 3653
    })
})

In [48]:
def formatting_func(example):
    source_text = example["src_text"]
    target_text = example["tgt_text"]

    model_inputs = tokenizer(
        source_text,
        truncation=True,
        max_length=512,
    )

    labels = tokenizer(
        target_text,
        truncation=True,
        max_length=256,
    )

    if "language_tag" in example:
        example.pop("language_tag")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [49]:
tokenized_train = split_ds["train"].map(
    formatting_func,
    batched=True,  # use batched=True if you want faster processing
)

tokenized_train

Map:   0%|          | 0/69407 [00:00<?, ? examples/s]

Dataset({
    features: ['src_text', 'tgt_text', 'input', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 69407
})

In [50]:
tokenized_test = split_ds["train"].map(
    formatting_func,
    batched=True,  # use batched=True if you want faster processing
)
tokenized_test

Map:   0%|          | 0/69407 [00:00<?, ? examples/s]

Dataset({
    features: ['src_text', 'tgt_text', 'input', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 69407
})

In [51]:
import wandb

wandb_key = UserSecretsClient().get_secret("WANDB_API_KEY")

wandb.login(key=wandb_key)

wandb.init(project=WANDB_PROJECT)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: egbewaleyanmife (yanmife-work-1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [52]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

In [58]:
training_args = TrainingArguments(
    output_dir="./nllb-nigeria",       
    per_device_train_batch_size=8,     # adjust based on GPU memory
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,     # for effective larger batch
    num_train_epochs=50,                # adjust based on dataset size
    learning_rate=1e-4,
    
    lr_scheduler_type="cosine",  
    warmup_steps=10000,               # gradual warmup

    logging_steps=500,
    save_steps=500,
    eval_steps=500,
    eval_strategy="steps",       # evaluate every N steps
    save_total_limit=2,                # keep last 2 checkpoints
    fp16=True,                         # use mixed precision if GPU supports
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [62]:
trainer = Trainer(
    model=model,                   # your Aya model
    args=training_args,
    train_dataset=tokenized_train,  # tokenized MENYO-20k train split
    eval_dataset=tokenized_test,     # tokenized dev split
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],

)

/tmp/ipykernel_37/4186336362.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [63]:
!rm -rf /content/Yanmife/nllb-nigeria
!rm -rf /content/Yanmife/nllb-nigeria

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [64]:
import torch, gc
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,3.117500,2.832947
1000,2.627300,2.164513
